<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python**

In [7]:
%%writefile tp2.py
import sys
import os
import random
from threading import Thread
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validateArgs(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(min=MIN_MATRIX_SIZE, max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrixGenerator(size):
    matrixResult=[]
    for i in range(0, size):
        matrixRow=[]
        for j in range(0, size):
            matrixRow.append(random.randint(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrixResult.append(matrixRow)
    return matrixResult

def printMatrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuentialMult(size, matrix_a, matrix_b):
    matrixResult = []
    for i in range(0, size):
        rowResult = [None] * size
        for j in range(0,size):
            for k in range(0,size):
                rowResult[j] = matrix_a[i][k] * matrix_b[k][j]
        matrixResult.append(rowResult)
    return matrixResult

## Funcion que ejecuta el thread
def multiplyRow(size, matrix_a, matrix_b, rowResult, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            rowResult[j]= matrix_a[rowIndex][k]*matrix_b[k][j]

## Genera un thread por cada fila encargado de ejecutar la multiplicacion y generar la fila resultante
def concurrentMult(size, matrix_a, matrix_b):
    matrixResult = []
    threads = []

    for i in range(0, size):
        rowResult = [None] * size
        thread = Thread(target=multiplyRow, args=(size, matrix_a, matrix_b, rowResult, i))
        thread.start()
        threads.append(thread)
        matrixResult.append(rowResult)

    for thread in threads:
        thread.join()

    return matrixResult

def compareMatrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validateArgs(sys.argv)
    matrix_a = matrixGenerator(size)
    matrix_b = matrixGenerator(size)
    matrix_secuential = secuentialMult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrentMult(size,matrix_a,matrix_b)

    printMatrix("A", matrix_a)
    printMatrix("B", matrix_b)
    printMatrix("Secuencial", matrix_secuential)
    printMatrix("Concurrente", matrix_concurrent)

    ## Determinamos si las matrices son iguales
    resultCompare = "iguales" if compareMatrix(size, matrix_secuential, matrix_concurrent) else "distintas"
    print("Las matrices concurrente y secuencial son {compare}".format(compare=resultCompare))

main()

Overwriting tp2.py


In [8]:
!python3 tp2.py 1

El tamaño debe estar entre 5 y 20


In [9]:
!python3 tp2.py 7


Matrix: A
	3		-22		-29		12		-32		-12		4	
	11		-9		29		32		-32		2		-18	
	-9		7		20		-32		27		19		-27	
	28		-19		13		-17		-2		15		11	
	-3		28		-19		-10		-1		17		11	
	16		13		-22		23		11		-25		15	
	18		-21		24		7		-17		-29		-32	

Matrix: B
	24		9		4		10		16		-27		-3	
	-28		-31		21		30		8		5		26	
	2		24		23		19		26		19		-15	
	-21		-5		-18		-19		1		-19		15	
	2		32		22		21		5		22		18	
	24		13		-24		-28		4		-26		29	
	19		-24		-18		21		32		8		-18	

Matrix: Secuencial
	76		-96		-72		84		128		32		-72	
	-342		432		324		-378		-576		-144		324	
	-513		648		486		-567		-864		-216		486	
	209		-264		-198		231		352		88		-198	
	209		-264		-198		231		352		88		-198	
	285		-360		-270		315		480		120		-270	
	-608		768		576		-672		-1024		-256		576	

Matrix: Concurrente
	76		-96		-72		84		128		32		-72	
	-342		432		324		-378		-576		-144		324	
	-513		648		486		-567		-864		-216		486	
	209		-264		-198		231		352		88		-198	
	209		-264		-198		231		352		88		-198	
	285		-360		-270		315		480		120		-270	
	-608		768		576		